In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import random
import os

In [2]:
path = './8bit/'

In [3]:
img_list = sorted(glob.glob(path + 'image/*.png'))
mask_list = sorted(glob.glob(path + 'mask/*.png'))

print(len(img_list), len(mask_list))

6690 6690


In [4]:
# Preprocessing + Pad Values
mask_cv = []
img_cv = []
borderType = cv2.BORDER_CONSTANT
top, bottom, right, left = 80, 80, 80, 80
value = [0,0,0]

for i in range(len(mask_list)):
    mask = cv2.imread(mask_list[i], cv2.IMREAD_ANYDEPTH)
    mask = cv2.copyMakeBorder(mask, top, bottom, left, right, borderType, None, value) # Pad Values
    mask_cv.append(mask)

for i in range(len(img_list)):
    img = cv2.imread(img_list[i], cv2.IMREAD_ANYDEPTH)
    img = cv2.copyMakeBorder(img, top, bottom, left, right, borderType, None, value) # Pad Values
    img_cv.append(img)
    

In [5]:
mask.shape

(672, 672)

In [6]:
# Crop by Center of Gravity

def Crop_CoG(img, mask, size):
    
    indices = np.where(mask != [0])
    y, x = indices[0], indices[1]
    a, b = (min(x), min(y)) # Lower Left Corner 
    W = (max(x) - min(x)) # Width  
    h = (max(y) - min(y)) # Height
    
    x = int(a+W/2)
    y = int(b+h/2)
    
    if size == 512:
        mask = mask[80:592, 80:592]
        img = img[80:592, 80:592]
        
        return img, mask
    
    else:
    
        mask = mask[int(y-size/2):int(y+size/2), int(x-size/2):int(x+size/2)]
        img = img[int(y-size/2):int(y+size/2), int(x-size/2):int(x+size/2)]
    
        return img, mask


In [7]:
size_list = [i for i in range(144, 321, 16)]
size_list.append(512)


In [8]:
a = len(str(path))
print(img_list[1000][a+6:a+16], mask_list[1000][a+5:a+15])

054_000048 054_000048


In [9]:
# Saving Image

for sizes in size_list:
    
    Image_dir = "./CenterCrop_" + path[2:] + str(sizes) +'/Image/'
    Mask_dir = "./CenterCrop_" + path[2:] + str(sizes) +'/Mask/'
    
    try:
        if not(os.path.isdir(Image_dir)):
            os.makedirs(os.path.join(Image_dir))
        if not(os.path.isdir(Mask_dir)):
            os.makedirs(os.path.join(Mask_dir))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise
    
    for i in range(len(img_list)):
    
        img_path = img_list[i][a+6:a+16]
        mask_path = mask_list[i][a+5:a+15]
    
        # Change Size and File Path
        img, mask = Crop_CoG(img_cv[i], mask_cv[i], sizes)
    
        cv2.imwrite(Image_dir + img_path + '.png', img)
        cv2.imwrite(Mask_dir + mask_path + '.png', mask)

    print('Finished size: ', sizes)
        

Finished size:  144
Finished size:  160
Finished size:  176
Finished size:  192
Finished size:  208
Finished size:  224
Finished size:  240
Finished size:  256
Finished size:  272
Finished size:  288
Finished size:  304
Finished size:  320
Finished size:  512
